# Juego de pong con un toque diferente
En este proyecto recrearemos el juego clasico de pong con algunas modificaciones para que sea mas divertido y a su vez siga patrones de diseño.



### Singleton
Utilizamos el patron singleton para configurar el tamaño de la pantalla y y el tamaño de los objetos del juego. Prevenimos que exista otra estancia de esta misma configuración

In [4]:
class GameConfig:
    # Declaramos una variable estatica privada que no puede ser modificada.
    __instance = None

    # Creamos un metodo estatico que nos permita acceder a esta instancia.
    @staticmethod
    def get_instance():
        if GameConfig.__instance is None:
            GameConfig()
        return GameConfig.__instance
    def __init__(self):
        # Declaramos las variables de configuracion del juego y aseguramos que no existan otras instancias de la clase.
        if GameConfig.__instance is not None:
            raise Exception("Esta clase es un singleton, no se puede crear otra instancia.")
        else:
            GameConfig.__instance = self
        
        # Configuracion del juego por defecto
        self.width = 1920
        self.height = 1080
        self.fps = 60
        self.ball_size = 10

    def set_width(self, width):
        self.width = width

    def set_height(self, height):
        self.height = height

    def set_fps(self, fps):
        self.fps = fps

    def set_ball_size(self, ball_size):
        self.ball_size = ball_size

### Flyweight y Command
Utilizamos el patron flyweight para reducir el uso de memoria con los sprites utilizados en el proyecto

In [5]:
import sys
import random
import math
import pygame

from typing import Dict, List, Tuple
from dataclasses import dataclass

from abc import ABC, abstractmethod

# Definimos las para la textura del juego ya que es constante y no cambia
@dataclass
class BallType:
    texture: str

# Definimos la clase Ball que representa una bola en el juego, con su textura y posición
@dataclass
class Ball:
    def __init__(self, ball_type: BallType, x: int, y: int, velocity: Tuple[int, int]):
        self.ball_type = ball_type
        self.x = x
        self.y = y
        self.velocity = velocity
        self.is_active = True

    # Actualizamos la posición de la bola en el juego
    def move(self, dx: int, dy: int):
        self.x += dx
        self.y += dy

    def get_position(self) -> Tuple[int, int]:
        return self.x, self.y

class BallManager:
    def __init__(self):
        self.balls: List[Ball] = []

    def add_ball(self, ball: Ball) -> None:
        self.balls.append(ball)

    def remove_ball(self, ball: Ball) -> None:
        self.balls.remove(ball)

    def move_balls(self) -> None:
        for ball in self.balls:
            if ball.is_active:
                ball.move(*ball.velocity)

# Clase commnad abstracta que define la interfaz para los comandos
class Command(ABC):
    @abstractmethod
    def execute(self) -> None:
        pass

# Clase PlayerType que maneja la textura de las paredes y su tamaño
class PlayerType:
    def __init__(self, texture: str, width: int):
        self.texture = texture
        self.width = width

# Clase Player que permite al usuario recibir comandos de movimiento
class Player:
    def __init__(self, x: int, y: int, player_type: PlayerType):
            self.x = x
            self.y = y
            self.player_type = player_type

    def move_left(self, distance: int) -> None:
        self.x -= distance
        if self.x < 0:
            self.x = 0
        
    def move_right(self, distance: int) -> None:
        self.x += distance
        if self.x > GameConfig.get_instance().width - self.player_type.width:
            self.x = GameConfig.get_instance().width - self.player_type.width

# Comandos concretos que implementan la interfaz Command
class MoveLeftCommand(Command):
    def __init__(self, player: Player, distance: int):
        self.player = player
        self.distance = distance

    def execute(self) -> None:
        self.player.move_left(self.distance)

class MoveRightCommand(Command):
    def __init__(self, player: Player, distance: int):
        self.player = player
        self.distance = distance

    def execute(self) -> None:
        self.player.move_right(self.distance)

# Clase InputHandler que maneja la entrada del usuario
class InputHandler:
    def __init__(self):
        self.commands: Dict[int, Command] = {}

    def bind_input(self, key: int, command: Command) -> None:
        self.commands[key] = command

    def handle_input(self, event) -> None:
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key in self.commands:
                self.commands[event.key].execute()            


### Inicializacion del juego con la libreria pygame

In [6]:
import pygame

from pygame.locals import QUIT, KEYDOWN, K_ESCAPE, K_UP, K_DOWN, K_LEFT, K_RIGHT

# Inicializamos la configuracion del juego
config = GameConfig.get_instance()

config.set_width(800)
config.set_height(600)
config.set_fps(30)
config.set_ball_size(5)

# Inicializamos pygame
pygame.init()

screen = pygame.display.set_mode((config.width, config.height))
pygame.display.set_caption("Configuracion del Juego")

# Creamos el jugador y la bola
player_type = PlayerType("player_texture.png", config.width // 10)
player = Player(config.width // 2, config.height // 10, player_type)
ball_type = BallType("ball_texture.png")
ball = Ball(ball_type, config.width // 2, config.height // 2, (random.choice([-1, 1]), random.choice([-1, 1])))

# Asignamos los comandos de movimiento al jugador
input_handler = InputHandler()
input_handler.bind_input(K_LEFT, MoveLeftCommand(player, player.player_type.width//2))
input_handler.bind_input(K_RIGHT, MoveRightCommand(player, player.player_type.width//2))
input_handler.bind_input(K_UP, MoveLeftCommand(player, player.player_type.width//2))
input_handler.bind_input(K_DOWN, MoveRightCommand(player, player.player_type.width//2))

# Creamos el ball manager y añadimos la bola inicial
ball_manager = BallManager()
ball_manager.add_ball(ball)

# Game loop
running = True
while running:
    # Limitamos la tasa de refresco
    pygame.time.Clock().tick(config.fps)
    for event in pygame.event.get():
        input_handler.handle_input(event)
        if event.type == QUIT:
            running = False
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False
    
    # Pintamos la pantalla a la tasa de refresco
    screen.fill((0, 0, 0))

    # Dibujamos el jugador y la bola
    pygame.draw.rect(screen, (255, 255, 255), (player.x, player.y, player.player_type.width, player.player_type.width//4))
    pygame.draw.circle(screen, (0, 255, 0), (ball.x, ball.y), config.ball_size)

    # Movemos las bolas
    ball_manager.move_balls()

    # Actualizamos la pantalla
    pygame.display.flip()
    
pygame.quit()

SystemExit: 